In [1]:
from isochrones import get_ichrone

mist = get_ichrone('mist')

In [134]:
import pandas as pd
import os
import glob
import tarfile

from isochrones.config import ISOCHRONES
from isochrones.utils import download_file
from isochrones.interp import DFInterpolator


class Grid(object):
    is_full = False

    def __init__(self, **kwargs):
        
        if hasattr(self, 'default_kwargs'):
            self.kwargs = self.default_kwargs.copy()
        else:
            self.kwargs = {}
        self.kwargs.update(kwargs)        

        self._df = None
        self._interp = None

    @property
    def datadir(self):
        raise NotImplementedError
        
    def get_hdf_filename(self, **kwargs):
        raise NotImplementedError

    @property
    def hdf_filename(self):
        return self.get_hdf_filename()
        
    def get_tarball_url(self, **kwargs):
        raise NotImplementedError
    
    def get_tarball_file(self, **kwargs):
        raise NotImplementedError

    def download_tarball(self, **kwargs):
        if not os.path.exists(self.datadir):
            os.makedirs(self.datadir)
        tarball = self.get_tarball_file(**kwargs)
        if not os.path.exists(tarball):
            url = self.get_tarball_url(**kwargs)
            logging.info('Downloading {}...'.format(url))
            download_file(url, tarball)    
        
    def extract_tarball(self, **kwargs):
        tarball = self.get_tarball_file(**kwargs)
        if not os.path.exists(tarball):
            self.download_tarball(**kwargs)

        with tarfile.open(tarball) as tar:
            logging.info('Extracting {}...'.format(tarball))
            tar.extractall(self.datadir)

    def read_hdf(self):
        h5file = self.hdf_filename
        try:
            df = pd.read_hdf(h5file, 'df')
        except FileNotFoundError:
            df = self.write_hdf()
        return df

    def write_hdf(self):
        df = self.get_df()
        h5file = self.hdf_filename
        df.to_hdf(h5file,'df')
        logging.info('{} written.'.format(h5file))
        return df

    @property
    def df(self):
        if self._df is None:
            self._df = self.get_df()
        return self._df
    
    @property
    def interp(self):
        if self._interp is None:
            self._interp = DFInterpolator(self.df, is_full=self.is_full)
        return self._interp

    
class BolometricCorrectionGrid(Grid):
    
    index_cols = ('Teff', 'logg', '[Fe/H]', 'Av', 'Rv')    
    name = None
    is_full = True
    
    def __init__(self, bands=None):

        self.bands = bands if bands is not None else list(self.default_bands)
    
        self._band_map = None
        self._phot_systems = None
        
        self._df = None
        self._interp = None
    
    def _make_band_map(self):
        phot_systems = set()
        band_map = {}
        for b in self.bands:
            phot, band = self.get_band(b)
            phot_systems.add(phot)
            band_map[b] = band
        self._band_map = band_map
        self._phot_systems = phot_systems
    
    @property
    def band_map(self):
        if self._band_map is None:
            self._make_band_map()
        return self._band_map

    @property
    def phot_systems(self):
        if self._phot_systems is None:
            self._make_band_map()
        return self._phot_systems
    
    @property
    def datadir(self):
        return os.path.join(ISOCHRONES, 'BC', self.name)
    
    def get_filename(self, phot, feh):
        rootdir = self.datadir
        sign_str = 'm' if feh < 0 else 'p'
        filename = 'feh{0}{1:03.0f}.{2}'.format(sign_str, abs(feh)*100, phot)
        return os.path.join(rootdir, filename)

    def parse_table(self, filename):
        """Reads text table into dataframe
        """
        with open(filename) as fin:
            for i, line in enumerate(fin):
                if i == 5:
                    names = line[1:].split()
                    break        
        return pd.read_csv(filename, names=names, delim_whitespace=True, comment='#',
                          index_col=self.index_cols)
                    
    def get_table(self, phot, feh):
        return self.parse_table(self.get_filename(phot, feh))
        
    def get_hdf_filename(self, phot):
        return os.path.join(self.datadir, '{}.h5'.format(phot))
        
    def get_tarball_url(self, phot):
        url = 'http://waps.cfa.harvard.edu/MIST/BC_tables/{}.txz'.format(phot)
        return url
    
    def get_tarball_file(self, phot):
        return os.path.join(self.datadir, '{}.txz'.format(phot))

    def get_df(self):  
        df_all = pd.DataFrame()
        for phot in self.phot_systems:
            hdf_filename = self.get_hdf_filename(phot=phot)
            if not os.path.exists(hdf_filename):
                filenames = glob.glob(os.path.join(self.datadir, '*.{}'.format(phot)))
                if not filenames:
                    self.extract_tarball(phot=phot)
                    filenames = glob.glob(os.path.join(self.datadir, '*.{}'.format(phot)))
                df = pd.concat([self.parse_table(f) for f in filenames]).sort_index()
                df.to_hdf(hdf_filename, 'df')
            df = pd.read_hdf(hdf_filename)
            df_all = pd.concat([df_all, df], axis=1)
            
        df_all = df_all.rename(columns={v:k for k, v in self.band_map.items()})
        for col in df_all.columns:
            if col not in self.bands:
                del df_all[col]
                
        return df_all
    
    
class MISTBolometricCorrectionGrid(BolometricCorrectionGrid):
    name = 'mist'
    
    phot_bands = dict(UBVRIplus=['Bessell_U', 'Bessell_B', 'Bessell_V',
                                 'Bessell_R', 'Bessell_I', '2MASS_J', '2MASS_H', '2MASS_Ks',
                                 'Kepler_Kp', 'Kepler_D51', 'Hipparcos_Hp',
                                 'Tycho_B', 'Tycho_V', 'Gaia_G_DR2Rev', 'Gaia_BP_DR2Rev', 
                                 'Gaia_RP_DR2Rev', 'TESS'],
                  WISE=['WISE_W1', 'WISE_W2', 'WISE_W3', 'WISE_W4'],
                  CFHT=['CFHT_u', 'CFHT_g', 'CFHT_r',
                        'CFHT_i_new', 'CFHT_i_old', 'CFHT_z'],
                  DECam=['DECam_u', 'DECam_g', 'DECam_r',
                         'DECam_i', 'DECam_z', 'DECam_Y'],
                  GALEX=['GALEX_FUV', 'GALEX_NUV'],
                  JWST=['F070W', 'F090W', 'F115W', 'F140M',
                       'F150W2', 'F150W', 'F162M', 'F164N', 'F182M', 'F187N', 'F200W',
                       'F210M', 'F212N', 'F250M', 'F277W', 'F300M', 'F322W2', 'F323N',
                       'F335M', 'F356W', 'F360M', 'F405N', 'F410M', 'F430M', 'F444W',
                       'F460M', 'F466N', 'F470N', 'F480M'],
                  LSST=['LSST_u', 'LSST_g', 'LSST_r',
                        'LSST_i', 'LSST_z', 'LSST_y'],
                  PanSTARRS=['PS_g', 'PS_r', 'PS_i', 'PS_z',
                             'PS_y', 'PS_w', 'PS_open'],
                  SkyMapper=['SkyMapper_u', 'SkyMapper_v', 'SkyMapper_g',
                             'SkyMapper_r', 'SkyMapper_i', 'SkyMapper_z'],
                  SPITZER=['IRAC_3.6', 'IRAC_4.5', 'IRAC_5.8', 'IRAC_8.0'],
                  UKIDSS=['UKIDSS_Z', 'UKIDSS_Y', 'UKIDSS_J',
                            'UKIDSS_H', 'UKIDSS_K'],
                  SDSS=['SDSS_u', 'SDSS_g', 'SDSS_r', 'SDSS_i', 'SDSS_z'])
            
    default_bands = ('J', 'H', 'K', 'G', 'BP', 'RP', 'W1', 'W2', 'W3', 'TESS', 'Kepler')
    
    def get_df(self, *args, **kwargs):
        df = super().get_df(*args, **kwargs)
        return df.xs(3.1, level='Rv')
    
    @classmethod
    def get_band(cls, b, **kwargs):
        """Defines what a "shortcut" band name refers to.  Returns phot_system, band

        """
        phot = None

        # Default to SDSS for these
        if b in ['u','g','r','i','z']:
            phot = 'SDSS'
            band = 'SDSS_{}'.format(b)
        elif b in ['U','B','V','R','I']:
            phot = 'UBVRIplus'
            band = 'Bessell_{}'.format(b)
        elif b in  ['J','H','Ks']:
            phot = 'UBVRIplus'
            band = '2MASS_{}'.format(b)
        elif b=='K':
            phot = 'UBVRIplus'
            band = '2MASS_Ks'
        elif b in ['kep','Kepler','Kp']:
            phot = 'UBVRIplus'
            band = 'Kepler_Kp'
        elif b=='TESS':
            phot = 'UBVRIplus'
            band = 'TESS'
        elif b in ['W1','W2','W3','W4']:
            phot = 'WISE'
            band = 'WISE_{}'.format(b)
        elif b in ('G', 'BP', 'RP'):
            phot = 'UBVRIplus'
            band = 'Gaia_{}_DR2Rev'.format(b)
            if 'version' in kwargs:
                if kwargs['version'] in ('1.1', '1.2'):
                    band += '_DR2Rev'
        else:
            m = re.match('([a-zA-Z]+)_([a-zA-Z_]+)',b)
            if m:
                if m.group(1) in cls.phot_bands.keys():
                    phot = m.group(1)
                    if phot=='PanSTARRS':
                        band = 'PS_{}'.format(m.group(2))
                    else:
                        band = m.group(0)
                elif m.group(1) in ['UK','UKIRT']:
                    phot = 'UKIDSS'
                    band = 'UKIDSS_{}'.format(m.group(2))

        if phot is None:
            for system, bands in cls.phot_bands.items():
                if b in bands:
                    phot = system
                    band = b
                    break
            if phot is None:
                raise ValueError('MIST grids cannot resolve band {}!'.format(b))
        return phot, band    

In [135]:
bc = MISTBolometricCorrectionGrid()

In [136]:
bc.df.head()

J         H         K    Kepler         G  \
Teff   logg [Fe/H] Av                                                       
2500.0 -4.0 -4.0   0.00  1.845781  2.927064  3.436304 -2.347335 -2.181986   
                   0.05  1.831466  2.917990  3.430463 -2.379940 -2.211637   
                   0.10  1.817153  2.908916  3.424623 -2.412506 -2.241240   
                   0.15  1.802841  2.899842  3.418782 -2.445036 -2.270797   
                   0.20  1.788530  2.890769  3.412942 -2.477527 -2.300306   

                               BP        RP      TESS        W1        W2  \
Teff   logg [Fe/H] Av                                                       
2500.0 -4.0 -4.0   0.00 -4.652544 -0.881255 -0.683335  3.612407  3.887990   
                   0.05 -4.697700 -0.909057 -0.709131  3.609537  3.886208   
                   0.10 -4.742838 -0.936829 -0.734897  3.606667  3.884427   
                   0.15 -4.787959 -0.964571 -0.760633  3.603797  3.882645   
                   0.20 -4.833062 -0.992285 -0.786338  3.600927  3.880864   

                               W3  
Teff   logg [Fe/H] Av              
2500.0 -4.0 -4.0   0.00  4.031450  
                   0.05  4.030978  
                   0.10  4.030505  
                   0.15  4.030033  
                   0.20  4.029561

In [137]:
bc.interp([5770., 4.44, 0.01, 0.0], ['G', 'BP', 'RP'])

array([ 0.08262917, -0.23317526,  0.5644952 ])

In [138]:
import os
import re
import itertools
import logging

import numpy as np
import pandas as pd

from isochrones.config import ISOCHRONES
from isochrones.interp import DFInterpolator
from isochrones.utils import download_file

class ModelGrid(Grid):

    default_columns = ('eep', 'age', 'feh', 'mass', 'initial_mass', 'radius',
                       'logTeff', 'Teff', 'logg', 'logL', 'Mbol')
    
        
    @property
    def prop_map(self):
        return dict(eep=self.eep_col, age=self.age_col, feh=self.feh_col,
                    mass=self.mass_col, initial_mass=self.initial_mass_col,
                    logTeff=self.logTeff_col, logg=self.logg_col, logL=self.logL_col)
    
    @property
    def column_map(self):
        return {v: k for k, v in self.prop_map.items()}
                
    @property
    def datadir(self):
        return os.path.join(ISOCHRONES, self.name)
        
    @property
    def kwarg_tag(self):
        raise NotImplementedError

    def get_directory_path(self, **kwargs):
        raise NotImplementedError
                    
    def get_existing_filenames(self, **kwargs):
        d = self.get_directory_path(**kwargs)
        if not os.path.exists(d):
            self.extract_tarball(**kwargs)
        return [os.path.join(d,f) for f in os.listdir(d) if re.search(self.filename_pattern, f)]

    def get_filenames(self, **kwargs):
        """ Returns list of all filenames corresponding to phot system and kwargs.
        """
        return self.get_existing_filenames(**kwargs)
    
    @classmethod
    def get_feh(cls, filename):
        raise NotImplementedError
            
    @classmethod
    def to_df(cls, filename):
        """Parse raw filename to dataframe
        """
        raise NotImplementedError
                
    def df_all(self):
        """Entire original model grid as dataframe
        
        TODO: also save this as HDF, in case it's useful for anything
        """
        df = pd.concat([self.to_df(f) for f in self.get_filenames()])
        df = df.sort_values(by=list(self.index_cols))
        df.index = [df[c] for c in self.index_cols]
        return df
    
    def compute_additional_columns(self, df):
        """
        """
        df['Teff'] = 10**df['logTeff']
        df['Mbol'] = 4.74 - 2.5 * df['logL']
        df['radius'] = 10**df['log_R']
        return df
    
    def get_df(self):
        """Returns column-mapped, pared-down, standardized version of model grid
        """
        df = self.df_all()
        df = df.rename(columns=self.column_map)
        df = self.compute_additional_columns(df)
        # Select only the columns we want
        df = df[list(self.default_columns)]
        return df
    
    @property
    def hdf_filename(self):
        return os.path.join(self.datadir, '{}{}.h5'.format(self.name, self.kwarg_tag))
        
    def get_dm_deep(self, compute=False):
        filename = os.path.join(self.datadir, 'dm_deep{}.h5'.format(self.kwarg_tag))

        compute = not os.path.exists(filename)

        if not compute:
            try:
                dm_deep = pd.read_hdf(filename, 'dm_deep')
            except:
                compute = True

        if compute:
            # need grid to work with first
            df = self.get_df()

            # Make bucket for derivative to go in
            df['dm_deep'] = np.nan

            # Compute derivative for each (feh, age) isochrone, and fill in
            for f,a in itertools.product(*df.index.levels[:2]):
                subdf = df.loc[f,a]
                deriv = np.gradient(np.gradient(subdf['initial_mass'], subdf['eep']))
                subdf.loc[:, 'dm_deep'] = deriv

            df.dm_deep.to_hdf(filename, 'dm_deep')
            dm_deep = pd.read_hdf(filename, 'dm_deep')

        return dm_deep    
    
    @property
    def df(self):
        if self._df is None:
            self._df = self.read_hdf()
            self._df['dm_deep'] = self.get_dm_deep()

        return self._df
    
    @property
    def interp_grid_npz_filename(self):
        return os.path.join(self.datadir, 'full_grid{}.npz'.format(self.kwarg_tag))
    
    @property
    def interp(self):
        if self._interp is None:
            self._interp = DFInterpolator(self.df, filename=self.interp_grid_npz_filename)
        return self._interp
    
class MISTModelGrid(ModelGrid):
    name = 'mist'
    
    eep_col = 'EEP'
    age_col = 'log10_isochrone_age_yr'
    feh_col = '[Fe/H]'
    mass_col = 'star_mass'
    initial_mass_col = 'initial_mass'
    logTeff_col = 'log_Teff'
    logg_col = 'log_g'
    logL_col = 'log_L'
    
    default_kwargs = {'version':'1.2', 'vvcrit':0.0, 'kind':'full_isos'}
    default_columns = ('eep', 'age', 'feh', 'mass', 'initial_mass', 'radius',
                       'logTeff', 'Teff', 'logg', 'logL', 'Mbol', 'delta_nu', 'nu_max', 'phase')

    @property
    def kwarg_tag(self):
        return '_v{version}_vvcrit{vvcrit}'.format(**self.kwargs)
       
    def compute_additional_columns(self, df):
        """
        """
        df['Teff'] = 10**df['logTeff']
        df['Mbol'] = 4.74 - 2.5 * df['logL']
        df['radius'] = 10**df['log_R']
        df['feh'] = df['log_surf_z'] - np.log10(df['surface_h1']) - np.log10(0.0181)  # Aaron Dotter told me so
        return df

        
class MISTIsochroneGrid(MISTModelGrid):

    name = 'mist'
    
    eep_col = 'EEP'
    age_col = 'log10_isochrone_age_yr'
    feh_col = '[Fe/H]'
    mass_col = 'star_mass'
    initial_mass_col = 'initial_mass'
    logTeff_col = 'log_Teff'
    logg_col = 'log_g'
    logL_col = 'log_L'
    
    default_kwargs = {'version':'1.2', 'vvcrit': 0.4, 'kind':'full_isos'}
    index_cols = ('log10_isochrone_age_yr', 'feh', 'EEP')
    
    default_columns = ('eep', 'age', 'feh', 'mass', 'initial_mass', 'radius',
                       'logTeff', 'Teff', 'logg', 'logL', 'Mbol', 'delta_nu', 'nu_max', 'phase')
    
    filename_pattern = '\.iso'
    
    @property
    def kwarg_tag(self):
        tag = super().kwarg_tag
        return '{tag}_{kind}'.format(tag=tag, **self.kwargs)

    def get_directory_path(self, **kwargs):
        return os.path.join(self.datadir, 'MIST{}'.format(self.kwarg_tag))

    def get_tarball_file(self, **kwargs):
        filename = self.get_directory_path(**kwargs)
        return '{}.tar.gz'.format(filename)
        
    def get_tarball_url(self, **kwargs):
        return 'http://waps.cfa.harvard.edu/MIST/data/tarballs_v{version}/MIST_v{version}_vvcrit{vvcrit}_{kind}.tar.gz'.format(**self.kwargs)

    @classmethod
    def get_feh(cls, filename):
        m = re.search('feh_([mp])([0-9]\.[0-9]{2})_afe', filename)
        if m:
            sign = 1 if m.group(1)=='p' else -1
            return float(m.group(2)) * sign
        else:
            raise ValueError('{} not a valid MIST file? Cannnot parse [Fe/H]'.format(filename))

    @classmethod
    def to_df(cls, filename):
        with open(filename, 'r', encoding='latin-1') as fin:
            while True:
                line = fin.readline()
                if re.match('# EEP', line):
                    column_names = line[1:].split()
                    break
        feh = cls.get_feh(filename)
        df = pd.read_table(filename, comment='#', delim_whitespace=True,
                             skip_blank_lines=True, names=column_names)
        df['feh'] = feh
        return df

    
class MISTBasicIsochroneGrid(MISTIsochroneGrid):

    default_kwargs = {'version':'1.2', 'vvcrit': 0.4, 'kind':'basic_isos'}


* Read full original dataframe from disk (saved as hdf)
* Convert using column_map to standard set of columns &rarr; this becomes self.df (which will have no extra columns)
* This dataframe is what is passed to .interp, the filled version of which is then saved to .npy on disk
* The .npy file should contain a 'columns' entry, that needs to be compared with self.df.columns on load
* OR... the .npz file should just have a name that is the hash of the name + kwarg_tag + columns?

In [139]:
grid = MISTIsochroneGrid()

In [140]:
grid.df.head()

eep  age       feh      mass  initial_mass  \
log10_isochrone_age_yr feh  EEP                                               
5.0                    -4.0 38    38  5.0 -3.978406  0.107656      0.107656   
                            39    39  5.0 -3.978406  0.111838      0.111838   
                            40    40  5.0 -3.978406  0.115990      0.115990   
                            41    41  5.0 -3.978406  0.120116      0.120116   
                            42    42  5.0 -3.978406  0.124217      0.124217   

                                   radius   logTeff         Teff      logg  \
log10_isochrone_age_yr feh  EEP                                              
5.0                    -4.0 38   1.150726  3.619738  4166.177329  3.343167   
                            39   1.175799  3.621226  4180.474965  3.339170   
                            40   1.201154  3.622701  4194.696380  3.335264   
                            41   1.226788  3.624164  4208.852666  3.331453   
                            42   1.252712  3.625616  4222.947800  3.327732   

                                     logL      Mbol   delta_nu      nu_max  \
log10_isochrone_age_yr feh  EEP                                              
5.0                    -4.0 38  -0.444460  5.851149  37.297680  297.014374   
                            39  -0.419786  5.789465  36.877928  295.351156   
                            40  -0.395355  5.728387  36.429349  293.405666   
                            41  -0.371160  5.667901  35.950745  291.162452   
                            42  -0.347189  5.607972  35.440645  288.604096   

                                 phase   dm_deep  
log10_isochrone_age_yr feh  EEP                   
5.0                    -4.0 38    -1.0 -0.000014  
                            39    -1.0 -0.000021  
                            40    -1.0 -0.000027  
                            41    -1.0 -0.000025  
                            42    -1.0 -0.000026

In [144]:
grid.interp([8.11, -1.13, 300.], ['initial_mass', 'Teff'])

array([1.68967850e+00, 1.06573450e+04, 4.02760709e+00])

In [142]:
! say "Computation complete!"

In [146]:
from isochrones.utils import download_file

def default_max_eep(mass):
    """For MIST v1.2
    """
    if mass < 0.6:
        return 454
    elif mass == 0.6: 
        return 605
    elif mass == 0.65:
        return 808
    elif mass < 6.0:
        return 1710
    else:
        return 808
    
def max_eep(mass, feh):
    """For MIST v1.2
    """
    eep = None
    if feh == -4.0:
        if mass < 0.6:
            eep = 454
        elif mass <= 0.94:
            eep = 631
        elif mass < 3.8:
            eep = 808
        elif mass <= 4.4:
            eep = 1409
        elif mass >= 18:
            eep = 631
    elif feh == -3.5:
        if mass == 0.65:
            eep = 631
        elif 0.65 < mass < 1.78:
            eep = 808
        elif mass == 1.78:
            eep = 1409
        elif 1.78 < mass <= 3.4:
            eep = 808
        elif mass >= 19:
            eep = 707
    elif feh == -3.0:
        if 0.7 <= mass <= 2.48:
            eep = 808
        elif 2.5 <= mass <= 4.4:
            eep = 1409
    elif feh == -2.5:
        if 0.7 <= mass <= 2.32:
            eep = 808
        elif 2.32 < mass <= 5.8:
            eep = 1409
    elif feh == 0.5:
        if 0.7 <= mass <= 0.75:
            eep = 808
            
    if eep is None:
        return default_max_eep(mass)
    else:
        return eep
    

class MISTEvolutionTrackGrid(MISTModelGrid):
    default_kwargs = {'version': '1.2', 'vvcrit': 0.4, 'afe': 0.0}
    
    fehs = (-4.00, -3.50, -3.00, -2.50, -2.00,
            -1.75, -1.50, -1.25, -1.00, -0.75, -0.50,
            -0.25, 0.00, 0.25, 0.50)

    index_cols = ('initial_feh', 'initial_mass', 'EEP')


    default_columns = ('eep', 'log_age', 'star_age', 'feh', 'initial_feh', 'mass', 'initial_mass', 'radius',
                       'logTeff', 'Teff', 'logg', 'logL', 'Mbol', 'delta_nu', 'nu_max', 'phase', 'interpolated')
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._masses = None
    
    @property
    def masses(self):
        if self._masses is None:
            self._masses = self.df.index.levels[1]
    
    @property
    def datadir(self):
        return os.path.join(ISOCHRONES, self.name, 'tracks')

    @property
    def kwarg_tag(self):
        return '_v{version}_vvcrit{vvcrit}'.format(**self.kwargs)

    @property
    def prop_map(self):
        return dict(eep=self.eep_col, 
                    mass=self.mass_col, initial_mass=self.initial_mass_col,
                    logTeff=self.logTeff_col, logg=self.logg_col, logL=self.logL_col)
    
    def compute_additional_columns(self, df):
        """
        """
        df = super().compute_additional_columns(df)
        df['log_age'] = np.log10(df['star_age'])
        return df
    
    def get_file_basename(self, feh):
        feh_sign = 'm' if feh < 0 else 'p'
        afe = self.kwargs['afe']
        afe_sign = 'm' if afe < 0 else 'p'
        fmt_dict = self.kwargs.copy()
        fmt_dict.update(dict(feh=abs(feh), feh_sign=feh_sign, afe_sign=afe_sign, afe=abs(self.kwargs['afe'])))
        return 'MIST_v{version}_feh_{feh_sign}{feh:.2f}_afe_{afe_sign}{afe:.1f}_vvcrit{vvcrit:.1f}_EEPS'.format(**fmt_dict)    
    
    def get_directory_path(self, feh):
        basename = self.get_file_basename(feh)
        return os.path.join(self.datadir, basename)
    
    def get_tarball_url(self, feh):
        basename = self.get_file_basename(feh)
        version = self.kwargs['version']
        return 'http://waps.cfa.harvard.edu/MIST/data/tarballs_v{version}/{basename}.tar.gz'.format(version=version, basename=basename)

    def get_tarball_filename(self, feh):
        basename = self.get_file_basename(feh)
        return os.path.join(self.datadir, '{}.tar.gz'.format(basename))
            
    def download_and_extract_all(self):
        for feh in self.fehs:
            self.extract_tarball(feh)
        
    @classmethod
    def get_mass(cls, filename):
        m = re.search('(\d{5})M.track.eep', filename)
        if m:
            return float(m.group(1))/100.
        else:
            raise ValueError('Cannot parse mass from {}.'.format(filename))
            
    @classmethod
    def to_df(cls, filename):
        with open(filename, 'r', encoding='latin-1') as fin:
            while True:
                line = fin.readline()
                if re.match('^# EEPs', line):
                    line = line.split()
                    eep_first = int(line[2])
                    eep_last = int(line[-1])
                elif re.match('#\s+ star_age', line):
                    column_names = line[1:].split()
                    break
        initial_mass = cls.get_mass(filename)
        df = pd.read_table(filename, comment='#', delim_whitespace=True,
                             skip_blank_lines=True, names=column_names)
        df['initial_mass'] = initial_mass
        try:
            df['EEP'] = np.arange(eep_first, eep_last+1, dtype=int)
        except ValueError:
            print('len(df) is {}; first, last eeps are {}, {} ({})'.format(len(df), eep_first, eep_last, filename))
        return df
    
    def get_feh_filenames(self, feh):
        directory = self.get_directory_path(feh)
        if not os.path.exists(directory):
            self.extract_tarball(feh)
        return glob.glob(os.path.join(directory, '*.track.eep'))
    
    def get_feh_hdf_filename(self, feh):
        directory = self.get_directory_path(feh)
        return os.path.join(directory, 'all_masses.h5')

    def get_feh_interpolated_hdf_filename(self, feh):
        directory = self.get_directory_path(feh)
        return os.path.join(directory, 'all_masses_interpolated.h5')

    def df_all_feh(self, feh):
        hdf_filename = self.get_feh_hdf_filename(feh)
        if os.path.exists(hdf_filename):
            df = pd.read_hdf(hdf_filename, 'df')
        else:
            df = pd.concat([self.to_df(f) for f in self.get_feh_filenames(feh)])
            df['initial_feh'] = feh
            df = df.sort_values(by=list(self.index_cols))
            df.index = [df[c] for c in self.index_cols]
            df.to_hdf(hdf_filename, 'df')
            df = pd.read_hdf(hdf_filename, 'df')
        return df

    def df_all_feh_interpolated(self, feh):
        """Same as df_all_feh but with missing track tails interpolated
        """
        hdf_filename = self.get_feh_interpolated_hdf_filename(feh)
        if os.path.exists(hdf_filename):
            df_interp = pd.read_hdf(hdf_filename, 'df')
        else:
            logging.info('Interpolating incomplete tracks for feh = {}'.format(feh))
            df = self.df_all_feh(feh)
            df_interp = df.copy()
            df_interp['interpolated'] = False
            masses = df.index.levels[1]
            for i, m in enumerate(masses):
                n_eep = len(df.xs(m, level='initial_mass'))
                eep_max = max_eep(m, feh)
                if not eep_max:
                    raise ValueError('No eep_max return value for ({}, {})?'.format(m, feh))
                if n_eep < eep_max:

                    # Find lower limit
                    ilo = i
                    found_lower = False
                    while not found_lower:
                        ilo -= 1
                        mlo = masses[ilo]
                        nlo = len(df.xs(mlo, level='initial_mass'))
                        if nlo >= eep_max:
                            found_lower = True
                        if ilo == 0:
                            raise ValueError('Did not find mlo for ({}, {})'.format(m, feh))

                    # Find upper limit
                    ihi = i
                    found_upper = False
                    while not found_upper:
                        ihi += 1
                        mhi = masses[ihi]
                        nhi = len(df.xs(mhi, level='initial_mass'))
                        if nhi >= eep_max:
                            found_upper = True
                        if ihi > len(masses):
                            raise ValueError('Did not find mhi for ({}, {})'.format(m, feh))

                    logging.info('{}: {} (expected {}).  Interpolating between {} and {}'.format(m, n_eep, eep_max, mlo, mhi))
                    new_eeps = np.arange(n_eep, eep_max + 1)
                    new_index = pd.MultiIndex.from_product([[feh], [m], new_eeps])
                    new_data = pd.DataFrame(index=new_index, columns=df_interp.columns, dtype=float)

                    # Interpolate values
                    norm_distance = (m - mlo) / (mhi - mlo)
                    lo_index = pd.MultiIndex.from_product([[feh], [mlo], new_eeps])
                    hi_index = pd.MultiIndex.from_product([[feh], [mhi], new_eeps])
                    new_data.loc[:, df.columns] = df.loc[lo_index, :].values * (1 - norm_distance) + df.loc[hi_index, :].values * norm_distance
                    new_data.loc[:, 'interpolated'] = True
                    df_interp = pd.concat([df_interp, new_data])
            
            df_interp.sort_index(inplace=True)
            df_interp.to_hdf(hdf_filename, 'df')
            df_interp = pd.read_hdf(hdf_filename, 'df')

        return df_interp
        
    def df_all(self):
        df = pd.concat([self.df_all_feh_interpolated(feh) for feh in self.fehs])
        return df
    
    @property
    def df(self):
        if self._df is None:
            self._df = self.read_hdf()
            self._df['dt_deep'] = self.get_dt_deep()

        return self._df
    
    def get_dt_deep(self, compute=False):
        filename = os.path.join(self.datadir, 'dt_deep{}.h5'.format(self.kwarg_tag))

        compute = not os.path.exists(filename)

        if not compute:
            try:
                dt_deep = pd.read_hdf(filename, 'dt_deep')
            except:
                compute = True

        if compute:
            # need grid to work with first
            df = self.get_df()

            # Make bucket for derivative to go in
            df['dt_deep'] = np.nan

            # Compute derivative for each (feh, age) isochrone, and fill in
            for f, m in itertools.product(*df.index.levels[:2]):
                subdf = df.loc[f, m]
                deriv = np.gradient(np.gradient(subdf['star_age'], subdf['eep']))
                subdf.loc[:, 'dt_deep'] = deriv

            df.dt_deep.to_hdf(filename, 'dt_deep')
            dt_deep = pd.read_hdf(filename, 'dt_deep')

        return dt_deep    
        

In [147]:
track_grid = MISTEvolutionTrackGrid()

In [148]:
pars = np.array([-0.04, 1.01, 353])
track_grid.interp(pars, ['Mbol', 'logg', 'Teff'])

array([4.54221886e+00, 4.40934251e+00, 5.92858362e+03])

In [85]:
# track_grid.download_and_extract_all()

In [149]:
class MISTIsochrone(object):
    
    grid_type = MISTIsochroneGrid
    bc_type = MISTBolometricCorrectionGrid
    _param_index_order = (1, 2, 0, 3, 4)  
    
    def __init__(self, bands=None):
        self.bands = bands if bands is not None else list(self.bc_type.default_bands)

        self._model_grid = None
        self._bc_grid = None
        
        self.param_index_order = list(self._param_index_order)
        
    @property
    def model_grid(self):
        if self._model_grid is None:
            self._model_grid = self.grid_type()
        return self._model_grid
        
    @property
    def bc_grid(self):
        if self._bc_grid is None:
            self._bc_grid = self.bc_type(self.bands)
        return self._bc_grid
        
    def interp_value(self, pars, props):
        """
        
        pars : age, feh, eep, [distance, AV]
        """
        pars = np.atleast_1d(pars[self.param_index_order])
        return self.model_grid.interp(pars, props)

    def interp_mag(self, pars, bands):
        """
        
        pars : age, feh, eep, distance, AV
        """
        i_bands = [self.bc_grid.interp.columns.index(b) for b in bands]

        return interp_mag(pars, self.param_index_order, 
                          self.model_grid.interp.grid, 
                          self.model_grid.interp.column_index['Teff'], 
                          self.model_grid.interp.column_index['logg'],
                          self.model_grid.interp.column_index['feh'],
                          self.model_grid.interp.column_index['Mbol'],
                          *self.model_grid.interp.index_columns,
                          self.bc_grid.interp.grid, i_bands,
                          *self.bc_grid.interp.index_columns)
        
class MISTTrack(MISTIsochrone):
    grid_type = MISTEvolutionTrackGrid
    _param_index_order = (2, 0, 1, 3, 4)

    
from numba import jit
from math import log10
from isochrones.interp import interp_value_3d, interp_value_4d

@jit(nopython=True)
def interp_mag(pars, index_order, model_grid, i_Teff, i_logg, i_feh, i_Mbol, 
               model_ii0, model_ii1, model_ii2, 
               bc_grid, bc_cols, 
               bc_ii0, bc_ii1, bc_ii2, bc_ii3):
    
    # logTeff, logg, logL returned.
    ipar0 = index_order[0]
    ipar1 = index_order[1]
    ipar2 = index_order[2]
    star_props = interp_value_3d(pars[ipar0], pars[ipar1], pars[ipar2],
                                 model_grid, [i_Teff, i_logg, i_feh, i_Mbol],
                                 model_ii0, model_ii1, model_ii2)
    Teff = star_props[0]
    logg = star_props[1]
    feh = star_props[2]
    AV = pars[4]
    bc = interp_value_4d(Teff, logg, feh, AV, 
                         bc_grid, bc_cols,
                         bc_ii0, bc_ii1, bc_ii2, bc_ii3)
    
    mBol = star_props[2]
    dist_mod = 5 * log10(pars[3]/10)
    
    mags = np.empty(len(bc))
    for i in range(len(bc)):
        mags[i] = mBol + dist_mod - bc[i]
    
    return Teff, logg, mags

In [150]:
from math import pi, log, sqrt
LOG_ONE_OVER_ROOT_2PI = log(1./sqrt(2*pi))

@jit(nopython=True)
def star_lnlike(pars, 
                spec_vals, spec_uncs,
                mag_vals, mag_uncs, i_mags,
                model_grid, i_logT, i_logg, i_logL, model_ii0, model_ii1, model_ii2, 
                bc_grid, bc_ii0, bc_ii1, bc_ii2, bc_ii3):

    Teff, logg, mags = interp_mag(pars, model_grid, i_logT, i_logg, i_logL, 
                                  model_ii0, model_ii1, model_ii2, 
                                  bc_grid, i_mags, bc_ii0, bc_ii1, bc_ii2, bc_ii3)
    
    lnlike = 0
    
    # Spec_vals are Teff, logg, feh
    val = spec_vals[0]
    unc = spec_uncs[0]
    model_val = Teff
    resid = (val - model_val)
    lnlike += LOG_ONE_OVER_ROOT_2PI + log(unc) - 0.5 * resid * resid / (unc * unc)
    
    # logg
    val = spec_vals[1]
    unc = spec_uncs[1]
    model_val = logg
    resid = (val - model_val)
    lnlike += LOG_ONE_OVER_ROOT_2PI + log(unc) - 0.5 * resid * resid / (unc * unc)
        
    # feh
    val = spec_vals[2]
    unc = spec_uncs[2]
    model_val = pars[2]  # feh
    resid = (val - model_val)
    lnlike += LOG_ONE_OVER_ROOT_2PI + log(unc) - 0.5 * resid * resid / (unc * unc)
    
    for i in range(len(mag_vals)):
        val = mag_vals[i]
        unc = mag_uncs[i]
        model_val = mags[i]
        lnlike += LOG_ONE_OVER_ROOT_2PI + log(unc) - 0.5 * resid * resid / (unc * unc)
        
    return lnlike




In [151]:
ic = MISTIsochrone()

In [152]:
pars = np.array([300., 9.5, 0.1, 200, 0.2])
bands = ['G', 'BP', 'RP']
ic.interp_mag(pars, bands)

(4185.087791021258,
 4.660144730008982,
 array([7.11836734, 7.89965972, 6.27215866]))

In [153]:
%timeit ic.interp_value(pars, ['logg', 'Teff'])
%timeit ic.interp_mag(pars, bands)

The slowest run took 6.41 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 10.8 µs per loop
10000 loops, best of 3: 40.6 µs per loop


In [154]:
ic2 = MISTIsochrone_Track()

In [155]:
pars2 = np.array([1.01, 353, 0.11, 200, 0.2])
ic2.interp_mag(pars2, bands)

(5733.612979290512,
 4.424850980556096,
 array([6.69414488, 7.06039213, 6.16097197]))

In [156]:
%timeit ic2.interp_value(pars2, ['logg', 'Teff', 'feh'])
%timeit ic2.interp_mag(pars2, bands)

The slowest run took 6.02 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 10.8 µs per loop
10000 loops, best of 3: 39.6 µs per loop


In [42]:
from isochrones import StarModel
from isochrones.priors import (salpeter_prior, feh_prior, q_prior,
                               age_prior, distance_prior, AV_prior, 
                               FlatPrior)

class BasicStarModel(StarModel):
#     _not_a_band = ('RA','dec','ra','Dec','maxAV','parallax','AV',
#                   'logg','Teff','feh','density', 'separation',
#                   'PA','resolution','relative','N','index', 'id')
    
    def __init__(self, ic, **kwargs):
        self._ic = ic
        self.kwargs = kwargs

        self.bands = [k for k in kwargs if k in self.ic.bc_grid.bands]
        self.props = [k for k in kwargs if k in self._not_a_band]
                
        self.spec_props = [self.kwargs[k] for k in ['Teff', 'logg', 'feh']]
        
        self._priors = {'mass':salpeter_prior,
                        'feh':feh_prior,
                        'q':q_prior,
                        'age':age_prior,
                        'distance':distance_prior,
                        'AV':AV_prior,
                        'eep':FlatPrior(bounds=(self.ic.mineep, self.ic.maxeep))}

        self._bounds = {'mass':None,
                        'feh':None,
                        'age':None,
                        'q':q_prior.bounds,
                        'distance':distance_prior.bounds,
                        'AV':AV_prior.bounds,
                        'eep':(self.ic.mineep, self.ic.maxeep)}

        if 'maxAV' in kwargs:
            self.set_bounds(AV=(0, kwargs['maxAV']))

        if 'max_distance' in kwargs:
            self.set_bounds(distance=(0, kwargs['max_distance']))
            
    def bounds(self, prop):
        if self._bounds[prop] is not None:
            return self._bounds[prop]
        elif prop=='mass':
            lo, hi = (self.ic.minmass, self.ic.maxmass)
            self._bounds['mass'] = (lo, hi)
            self._priors['mass'].bounds = (lo, hi)
        elif prop=='feh':
            lo, hi = (self.ic.minfeh, self.ic.maxfeh)
            self._bounds['feh'] = (lo, hi)
            self._priors['feh'].bounds = (lo, hi)
        elif prop=='age':
            lo, hi = (self.ic.minage, self.ic.maxage)
            self._bounds['age'] = (lo, hi)
            self._priors['age'].bounds = (lo, hi)
            self._bounds['age'] = (self.ic.minage,
                                   self.ic.maxage)
        else:
            raise ValueError('Unknown property {}'.format(prop))
        return self._bounds[prop]

    def set_bounds(self, **kwargs):
        for k,v in kwargs.items():
            if len(v) != 2:
                raise ValueError('Must provide (min, max)')
            self._bounds[k] = v
            self._priors[k].bounds = v

    def lnlike(self, pars):
        spec_vals = [self.kwargs[prop][0] for prop in ['Teff', 'logg', 'feh']]
        spec_uncs = [self.kwargs[prop][1] for prop in ['Teff', 'logg', 'feh']]
        mag_vals = [self.kwargs[b][0] for b in self.bands]
        mag_uncs = [self.kwargs[b][1] for b in self.bands]
        i_mags = [self.ic.bc_grid.interp.column_index[b] for b in self.bands]
        
        return star_lnlike(pars, spec_vals, spec_uncs,
                           mag_vals, mag_uncs, i_mags, 
                           self.ic.model_grid.interp.grid, 10, 12, 7,
                           *self.ic.model_grid.interp.index_columns,
                           self.ic.bc_grid.interp.grid, 
                           *self.ic.bc_grid.interp.index_columns)
    
    def lnprior(self, pars):
        lnp = 0
        
        # Get prior for EEP:  p(EEP) = p(M) * d(M)/d(EEP)

In [51]:
%load_ext line_profiler

In [56]:
%lprun -f MISTIsochrone.interp_value ic2.interp_value(pars2, ['logg', 'Teff', 'feh'])

Timer unit: 1e-06 s

Total time: 0.001681 s
File: <ipython-input-40-3ad5a92c2310>
Function: interp_value at line 27

Line #      Hits         Time  Per Hit   % Time  Line Contents
    27                                               def interp_value(self, pars, props):
    28                                                   """
    29                                                   
    30                                                   pars : age, feh, eep, [distance, AV]
    31                                                   """
    32         1         86.0     86.0      5.1          pars = np.atleast_1d(pars[self.param_index_order])
    33         1       1595.0   1595.0     94.9          return self.model_grid.interp(pars, props)

In [47]:
%timeit ic2.interp_value(pars2, ['logg', 'Teff'])

The slowest run took 5.22 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 12 µs per loop


In [57]:
mod = BasicStarModel(ic, Teff=(5700., 50.), logg=(4.44, 0.1), feh=(0.0, 0.1), G=(9.71, 0.01), BP=(9.85, 0.01))

AttributeError: 'MISTIsochrone' object has no attribute 'mineep'

In [97]:
pars = np.array([300, 8.54, -0.4, 200., 0.1])

In [98]:
mod.lnlike(pars)

-2250.4050015378325

In [99]:
%timeit mod.lnlike(pars)

10000 loops, best of 3: 54.3 µs per loop


In [90]:
mod.lnpost(pars)

AttributeError: 'BasicStarModel' object has no attribute 'obs'

In [ ]:
ic.model_grid.interp.column_index

In [56]:
ic.model_grid.df.columns

Index(['EEP', 'log10_isochrone_age_yr', 'initial_mass', 'star_mass',
       'star_mdot', 'he_core_mass', 'c_core_mass', 'log_L', 'log_LH',
       'log_LHe', 'log_Teff', 'log_R', 'log_g', 'surface_h1', 'surface_he3',
       'surface_he4', 'surface_c12', 'surface_o16', 'log_center_T',
       'log_center_Rho', 'center_gamma', 'center_h1', 'center_he4',
       'center_c12', 'phase', 'feh', 'dm_deep'],
      dtype='object')

In [54]:
pars = np.array([8.54, -0.4, 300., 200., 0.1])

In [55]:
%timeit ic.interp_mag(pars, ['G', 'BP', 'RP', 'J', 'H', 'K'])

The slowest run took 33873.93 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 33.7 µs per loop


In [56]:
ic.bc_grid.bands

['J', 'H', 'K', 'G', 'TESS', 'BP']

In [59]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [61]:
%lprun -f ic.interp_mag ic.interp_mag(np.array([8.54, -0.4, 300., 200., 0.1]), ['G', 'BP'])

Timer unit: 1e-06 s

Total time: 0.00012 s
File: <ipython-input-52-2b6bb4589e2b>
Function: interp_mag at line 32

Line #      Hits         Time  Per Hit   % Time  Line Contents
    32                                               def interp_mag(self, pars, bands):
    33                                                   """
    34                                                   
    35                                                   pars : age, feh, eep, distance, AV
    36                                                   """
    37         1         55.0     55.0     45.8          logTeff, logg, logL = self.model_grid.interp(pars[:3], ['log_Teff', 'log_g', 'log_L'])
    38                                           
    39         1         11.0     11.0      9.2          bc_pars = np.array([10**logTeff, logg, pars[1], pars[4]])  # pars[0] is feh
    40         1         30.0     30.0     25.0          bc = self.bc_grid.interp(bc_pars, bands)        
    41                        

In [63]:
ic.interp_mag(np.array([8.54, -0.4, 300., 200., 0.1]), ['G', 'BP'])

(3.873409202781381, 4.369675032445283, array([9.71631876, 9.85953556]))

In [66]:
from isochrones import StarModel as StarModelOld
from isochrones import get_ichrone

mist = get_ichrone('mist')

mod_old = StarModelOld(mist, Teff=(5700, 50), logg=(4.44, 0.1), G=(9.71, 0.01), BP=(9.85, 0.01))

In [67]:
mod_old.lnlike([400, 9.6, 0.1, 200, 0.1])

KeyError: 'P'

In [49]:
mod.bands

['G']

In [50]:
mod.props

['Teff', 'logg']

In [51]:
mod.kwargs

{'Teff': (5700, 50), 'logg': (4.44, 0.1), 'G': (8.1, 0.01)}

In [5]:
from isochrones.interp import find_indices_3d, find_indices, find_indices_4d

In [6]:
col0 = ic.model_grid.interp.index_columns[0]
col1 = ic.model_grid.interp.index_columns[1]
col2 = ic.model_grid.interp.index_columns[2]
i1, d1, oob1 = find_indices([8.11, -1.1, 300.], ic.model_grid.interp.index_columns)
i2, d2, oob2 = find_indices_3d(8.11, -1.1, 300., col0, col1, col2)
assert (i1 == i2).all()
assert (d1 == d2).all()
assert oob1 == oob2
%timeit find_indices_3d(8.11, -1.1, 300., col0, col1, col2)

The slowest run took 4.48 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.62 µs per loop


In [7]:
col0 = ic.bc_grid.interp.index_columns[0]
col1 = ic.bc_grid.interp.index_columns[1]
col2 = ic.bc_grid.interp.index_columns[2]
col3 = ic.bc_grid.interp.index_columns[3]
i1, d1, oob1 = find_indices([5842., 4.34, 0.03, 0.42], ic.bc_grid.interp.index_columns)
i2, d2, oob2 = find_indices_4d(5842., 4.34, 0.03, 0.42, col0, col1, col2, col3)
assert (i1 == i2).all()
assert (d1 == d2).all()
assert oob1 == oob2
%timeit find_indices_4d(5842., 4.34, 0.03, 0.42, col0, col1, col2, col3)

The slowest run took 4.26 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.83 µs per loop


In [8]:
%timeit find_indices(np.array([5842., 4.34, 0.03, 0.42]), ic.bc_grid.interp.index_columns)

The slowest run took 53366.84 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.68 µs per loop


In [9]:
%timeit find_indices(np.array([8.11, -1.1, 300]), ic.model_grid.interp.index_columns)

The slowest run took 49952.27 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.48 µs per loop


In [12]:
%load_ext line_profiler

In [17]:
pars = np.array([8.11, -1.1, 300, 200, 0.])
bands = ['G', 'BP']
%timeit ic.interp_mag(pars, bands)

The slowest run took 5.29 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 28.1 µs per loop


In [16]:
%timeit ic.interp_value(pars, ['log_g', 'log_Teff', 'log_L'])

The slowest run took 6.77 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 7.88 µs per loop


In [15]:
%lprun -f ic.model_grid.interp.__call__ ic.interp_value(pars, ['log_g', 'log_Teff', 'log_L'])

Timer unit: 1e-06 s

Total time: 5.8e-05 s
File: /Users/tdm/repositories/isochrones/isochrones/interp.py
Function: __call__ at line 441

Line #      Hits         Time  Per Hit   % Time  Line Contents
   441                                               def __call__(self, p, cols):
   442         1         23.0     23.0     39.7          icols = np.array([self.column_index[col] for col in cols])
   443         1          3.0      3.0      5.2          args = (p, self.grid, icols, self.index_columns)
   444                                           
   445         1          2.0      2.0      3.4          if self.ndim == 3:
   446         1          3.0      3.0      5.2              args = (p[0], p[1], p[2], self.grid, icols,
   447         1          1.0      1.0      1.7                      self.index_columns[0], self.index_columns[1],
   448         1          1.0      1.0      1.7                      self.index_columns[2])
   449         1          3.0      3.0      5.2           

In [90]:
pars = np.array([8.11, -1.1, 300, 200, 0.])
bands = ['G', 'BP']
ic.interp_mag(pars, bands)

array([8.64000568, 8.60319518])

In [94]:
%timeit ic.interp_value(pars, ['log_g'])

The slowest run took 5.83 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 7.96 µs per loop


In [85]:
pars = np.array([8.11, -1.1, 300, 200, 1.1])
ic.interp_mag(pars, bands)

array([9.70095322, 9.85891442])

In [ ]:
ic.interp_value()

In [21]:
mist.mag['G'](300., 8.11, -1.1, 200., 0.)

array([8.64305202])

In [24]:
mist.mag['G'](300., 8.11, -1.1, 200., 1.1)

array([9.51726849])

In [18]:
bc.df.columns

Index(['Bessell_U', 'Bessell_B', 'Bessell_V', 'Bessell_R', 'Bessell_I',
       '2MASS_J', '2MASS_H', '2MASS_Ks', 'Kepler_Kp', 'Kepler_D51',
       'Hipparcos_Hp', 'Tycho_B', 'Tycho_V', 'Gaia_G_DR2Rev', 'Gaia_BP_DR2Rev',
       'Gaia_RP_DR2Rev', 'TESS'],
      dtype='object')

In [40]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [41]:
%lprun -f ic.interp_mag ic.interp_mag(pars, bands)

Timer unit: 1e-06 s

Total time: 0.000136 s
File: <ipython-input-26-365158820d11>
Function: interp_mag at line 14

Line #      Hits         Time  Per Hit   % Time  Line Contents
    14                                               def interp_mag(self, pars, bands):
    15                                                   """
    16                                                   
    17                                                   pars : age, feh, eep, distance, AV
    18                                                   """
    19         1         65.0     65.0     47.8          logTeff, logg, logL = self.models.interp(pars[:3], ['log_Teff', 'log_g', 'log_L'])
    20                                           
    21         1         13.0     13.0      9.6          bc_pars = np.array([10**logTeff, logg, pars[1], pars[4]])  # pars[0] is feh
    22         1         33.0     33.0     24.3          bc = self.bcs.interp(bc_pars, bands)        
    23                               

In [13]:
%timeit grid.interp([8.11, -1.1, 300.], ['log_L', 'initial_mass', 'star_mass', 'log_Teff'])

The slowest run took 4.86 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 15.5 µs per loop


In [84]:
grid.df.head()

EEP  log10_isochrone_age_yr  initial_mass  \
log10_isochrone_age_yr feh  EEP                                              
5.0                    -4.0 35    35                     5.0      0.100000   
                            36    36                     5.0      0.102885   
                            37    37                     5.0      0.107147   
                            38    38                     5.0      0.111379   
                            39    39                     5.0      0.115581   

                                 star_mass     star_mdot  he_core_mass  \
log10_isochrone_age_yr feh  EEP                                          
5.0                    -4.0 35    0.100000 -1.455094e-13           0.0   
                            36    0.102885 -1.562027e-13           0.0   
                            37    0.107147 -1.707298e-13           0.0   
                            38    0.111379 -1.836256e-13           0.0   
                            39    0.115581 -1.949639e-13           0.0   

                                 c_core_mass     log_L    log_LH  log_LHe  \
log10_isochrone_age_yr feh  EEP                                             
5.0                    -4.0 35           0.0 -0.489734 -1.922517    -99.0   
                            36           0.0 -0.472691 -1.878584    -99.0   
                            37           0.0 -0.447471 -1.814151    -99.0   
                            38           0.0 -0.422498 -1.749734    -99.0   
                            39           0.0 -0.397776 -1.685330    -99.0   

                                   ...      surface_o16  log_center_T  \
log10_isochrone_age_yr feh  EEP    ...                                  
5.0                    -4.0 35     ...     6.108882e-07      5.857643   
                            36     ...     6.108882e-07      5.861047   
                            37     ...     6.108882e-07      5.866071   
                            38     ...     6.108882e-07      5.871107   
                            39     ...     6.108882e-07      5.876158   

                                 log_center_Rho  center_gamma  center_h1  \
log10_isochrone_age_yr feh  EEP                                            
5.0                    -4.0 35        -0.170338      0.299672   0.750981   
                            36        -0.179318      0.295785   0.750981   
                            37        -0.192620      0.290038   0.750981   
                            38        -0.205703      0.284326   0.750981   
                            39        -0.218572      0.278647   0.750981   

                                 center_he4    center_c12  phase  feh  \
log10_isochrone_age_yr feh  EEP                                         
5.0                    -4.0 35     0.248961  2.496535e-07   -1.0 -4.0   
                            36     0.248961  2.496535e-07   -1.0 -4.0   
                            37     0.248961  2.496535e-07   -1.0 -4.0   
                            38     0.248961  2.496535e-07   -1.0 -4.0   
                            39     0.248961  2.496535e-07   -1.0 -4.0   

                                  dm_deep  
log10_isochrone_age_yr feh  EEP            
5.0                    -4.0 35   0.000688  
                            36   0.000681  
                            37   0.000322  
                            38  -0.000029  
                            39  -0.000027  

[5 rows x 27 columns]

In [73]:
len(grid.df)

1328838

In [56]:
df = grid.df_all()

In [57]:
df.head()

EEP  log10_isochrone_age_yr  initial_mass  \
log10_isochrone_age_yr feh  EEP                                              
5.0                    -4.0 35    35                     5.0      0.100000   
                            36    36                     5.0      0.102885   
                            37    37                     5.0      0.107147   
                            38    38                     5.0      0.111379   
                            39    39                     5.0      0.115581   

                                 star_mass     star_mdot  he_core_mass  \
log10_isochrone_age_yr feh  EEP                                          
5.0                    -4.0 35    0.100000 -1.455094e-13           0.0   
                            36    0.102885 -1.562027e-13           0.0   
                            37    0.107147 -1.707298e-13           0.0   
                            38    0.111379 -1.836256e-13           0.0   
                            39    0.115581 -1.949639e-13           0.0   

                                 c_core_mass     log_L    log_LH  log_LHe  \
log10_isochrone_age_yr feh  EEP                                             
5.0                    -4.0 35           0.0 -0.489734 -1.922517    -99.0   
                            36           0.0 -0.472691 -1.878584    -99.0   
                            37           0.0 -0.447471 -1.814151    -99.0   
                            38           0.0 -0.422498 -1.749734    -99.0   
                            39           0.0 -0.397776 -1.685330    -99.0   

                                ...    surface_c12   surface_o16  \
log10_isochrone_age_yr feh  EEP ...                                
5.0                    -4.0 35  ...   2.496535e-07  6.108882e-07   
                            36  ...   2.496535e-07  6.108882e-07   
                            37  ...   2.496535e-07  6.108882e-07   
                            38  ...   2.496535e-07  6.108882e-07   
                            39  ...   2.496535e-07  6.108882e-07   

                                 log_center_T  log_center_Rho  center_gamma  \
log10_isochrone_age_yr feh  EEP                                               
5.0                    -4.0 35       5.857643       -0.170338      0.299672   
                            36       5.861047       -0.179318      0.295785   
                            37       5.866071       -0.192620      0.290038   
                            38       5.871107       -0.205703      0.284326   
                            39       5.876158       -0.218572      0.278647   

                                 center_h1  center_he4    center_c12  phase  \
log10_isochrone_age_yr feh  EEP                                               
5.0                    -4.0 35    0.750981    0.248961  2.496535e-07   -1.0   
                            36    0.750981    0.248961  2.496535e-07   -1.0   
                            37    0.750981    0.248961  2.496535e-07   -1.0   
                            38    0.750981    0.248961  2.496535e-07   -1.0   
                            39    0.750981    0.248961  2.496535e-07   -1.0   

                                 feh  
log10_isochrone_age_yr feh  EEP       
5.0                    -4.0 35  -4.0  
                            36  -4.0  
                            37  -4.0  
                            38  -4.0  
                            39  -4.0  

[5 rows x 26 columns]

In [25]:
len(df)

1328838

In [18]:
grid.get_existing_filenames()

['/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m0.25_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m0.50_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m0.75_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m1.00_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m1.25_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m1.50_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m1.75_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m2.00_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m2.50_afe_p0.0_vvcrit0.0_basi

In [11]:
ls /Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_full_isos/MIST_v1.2_feh_m0.25_afe_p0.0_vvcrit0.0_full.iso

/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_full_isos/MIST_v1.2_feh_m0.25_afe_p0.0_vvcrit0.0_full.iso


In [14]:
grid = MISTModelGrid()

In [ ]:
df.to

In [23]:
df.head()

,EEP,log10_isochrone_age_yr,initial_mass,star_mass,star_mdot,he_core_mass,c_core_mass,o_core_mass,log_L,log_L_div_Ledd,...,nu_max,acoustic_cutoff,max_conv_vel_div_csound,max_gradT_div_grada,gradT_excess_alpha,min_Pgas_div_P,max_L_rad_div_Ledd,e_thermal,phase,feh
0,15,5.0,0.100000,0.100000,-9.296476e-14,0.0,0.0,0.0,-0.793690,-4.301081,...,210.053677,1426.188700,0.139220,1.105393,0.0,0.999989,0.000017,2.444569e+46,-1.0,-0.25
1,16,5.0,0.104585,0.104585,-1.045561e-13,0.0,0.0,0.0,-0.761784,-4.244876,...,207.195212,1404.410306,0.138068,1.110125,0.0,0.999988,0.000019,2.570014e+46,-1.0,-0.25
2,17,5.0,0.108845,0.108845,-1.146222e-13,0.0,0.0,0.0,-0.732385,-4.193214,...,204.470239,1383.919005,0.136990,1.114414,0.0,0.999986,0.000021,2.688820e+46,-1.0,-0.25
3,18,5.0,0.113102,0.113102,-1.239873e-13,0.0,0.0,0.0,-0.703268,-4.142182,...,201.678757,1363.203146,0.136310,1.118595,0.0,0.999985,0.000023,2.809798e+46,-1.0,-0.25
4,19,5.0,0.117363,0.117363,-1.327007e-13,0.0,0.0,0.0,-0.674348,-4.091646,...,198.795191,1342.099109,0.135199,1.122682,0.0,0.999984,0.000024,2.933236e+46,-1.0,-0.25


In [ ]:
grid.get_directory_path('full_isos')